In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.pool import NullPool
import matplotlib as plt
import matplotlib.pylab as plt
import seaborn as sns
from tqdm import tqdm
from PIL import Image
import requests
from io import BytesIO



In [122]:
import requests
from urllib import parse

def get_data(keyword):
    enc = parse.quote(keyword)
    url = f"http://172.31.151.225:5000/api/search/uat?keyword={enc}&page=1&size=200&age=0&gender=&accuracy="
    return requests.get(url=url).json()['products']

3월 24일 광고 신청된 키워드들

In [201]:
q1 = '''
SELECT a.keyword,
       a.bid_price,
       b.id as ad_set_product_id,
       b.ad_set_id,
       b.creator_uid AS UID
FROM service1_quicket.ad_set_product_bid_keyword a
JOIN service1_quicket.ad_set_product b ON a.ad_set_product_id = b.id
WHERE a.created_at BETWEEN '2021-03-24 00:00:00' AND '2021-03-24 23:59:59'
  AND b.status NOT IN (1,
                      2,
                      4,
                      6)
'''
q2 = '''
SELECT a.keyword,
       a.bid_price,
       b.id as ad_set_product_id,
       b.ad_set_id,
       b.creator_uid AS UID
FROM service1_quicket.ad_set_product_bid_keyword a
JOIN service1_quicket.ad_set_product b ON a.ad_set_product_id = b.id
WHERE b.status = 3
'''
# keywords = pd.read_sql(q1, con=bun_dw)
# keywords2 = pd.read_sql(q2, con=bun_dw) #  df_pre에 넣을 그당시 진행중 광고
keywords3 = pd.read_sql(q2, con=bun_dw) #  df_post "

In [174]:
keywords2.ad_set_product_id.nunique()

4066

In [119]:
keywords['ad_set_product_id'] = keywords['ad_set_product_id'].astype(str)

In [97]:
keywords.shape

(8492, 5)

In [98]:
keywords['keyword'].nunique()

4302

In [99]:
top_keywords = keywords.groupby('keyword', as_index=False).size().sort_values('size', ascending=False)[:1000]['keyword']

In [101]:
pid_keyword_cnt = keywords.groupby('ad_set_product_id', as_index=False)['keyword'].count().sort_values('keyword', ascending=False)

In [102]:
pid_keyword_cnt.columns = ['ad_set_product_id', 'keyword_cnt']

In [103]:
pid_keyword_cnt.describe()

,ad_set_product_id,keyword_cnt
count,1433.000000,1433.000000
mean,214214.707606,5.926029
std,856.916438,4.740188
min,194447.000000,1.000000
25%,213813.000000,3.000000
50%,214278.000000,5.000000
75%,214679.000000,7.000000
max,215072.000000,36.000000


In [104]:
keywords.groupby('keyword', as_index=False)['ad_set_product_id'].count().sort_values('ad_set_product_id', ascending=False)

,keyword,ad_set_product_id
994,나이키,64
2339,새상품,49
3768,컴퓨터,49
2954,애슐리상점,45
1370,데스크탑,41
...,...,...
1706,마더텅,1
1708,마르지엘라 독일군,1
1709,마르지엘라 맨투맨,1
1710,마르지엘라독일군페인팅,1


In [106]:
iphone12_ad_product_ids = keywords[keywords['keyword'] == '아이폰12']['ad_set_product_id']
keywords[keywords['ad_set_product_id'].isin(iphone12_ad_product_ids)].groupby('ad_set_product_id', as_index=False)['keyword'].count()

,ad_set_product_id,keyword
0,213935,19
1,213936,19
2,213955,5
3,213974,10
4,214099,8
5,214734,24
6,214891,3
7,214892,4
8,214893,4
9,214898,3


In [154]:
def get_rank(keyword):
    result = {}
    if keyword not in pre:
        return result
    ids = []
    for rank, item in pre[keyword].items():
        ids.append(item[1])
    
    if len(ids) < 1:
        return result
    
    ids = list(set(ids))
    for ad_set_product_id in ids:
        ad_set_product_id = str(ad_set_product_id)
        keyword_list = keywords[keywords['ad_set_product_id'] == ad_set_product_id]['keyword'].unique()
        for keyword in keyword_list:
            pre_rank = None
            post_rank = None
            if keyword in pre:
                for rank, item in pre[keyword].items():
                    if str(item[1]) == ad_set_product_id:
                        pre_rank = rank
                        break
            if keyword in post:
                for rank, item in post[keyword].items():
                    if str(item[1]) == ad_set_product_id:
                        post_rank = rank
                        break
            product_dict = result.get(ad_set_product_id, {})
            product_dict[keyword] = (pre_rank, post_rank)
            result[ad_set_product_id] = product_dict
                
    return result

In [141]:
get_rank('아이폰')

{'213463': {'강아지케이스': (None, None),
  '갤럭시케이스': (None, None),
  '고양이케이스': (None, None),
  '기념일': (None, None),
  '명품케이스 ': (None, None),
  '버즈라이브케이스': (None, None),
  '버즈케이스': (None, None),
  '아이폰케이스': (1, 0),
  '에어팟케이스': (1, 1),
  '주문제작케이스': (None, None),
  '커스텀': (1, 2),
  '커스텀케이스': (None, None),
  '커플': (None, None),
  '커플케이스': (None, None),
  '케이스': (None, None),
  '폰케이스 ': (None, None),
  '핸드폰케이스 ': (None, None),
  '휴대폰케이스 ': (None, None)},
 '213478': {'목걸이': (4, None),
  '무료배송': (1, None),
  '반지': (0, None),
  '실버': (None, None),
  '써지컬': (None, None),
  '아이폰': (2, None),
  '악세사리': (None, None),
  '체인': (None, None),
  '팔찌': (0, None)},
 '213586': {'12프로맥스': (1, 1), '아이폰12프로맥스': (4, 4)},
 '213939': {'아이폰6': (0, 0)},
 '213940': {'아이폰6s': (0, 0)},
 '213941': {'아이폰7': (3, 5),
  '아이폰7플러스': (3, 3),
  '아이폰8': (6, 8),
  '아이폰8플러스': (3, 4)},
 '213942': {'아이폰7': (4, 7),
  '아이폰7플러스': (4, 4),
  '아이폰8': (7, 10),
  '아이폰8플러스': (4, 5)},
 '213943': {'아이폰7': (1, 2),
  '아이폰7플러스': (1, 1),
  '아이폰8': 

In [142]:
get_rank('나이키')

{'213400': {'fog': (None, None),
  'nrg': (0, None),
  '에센셜': (2, None),
  '웜업': (None, None),
  '피오갓': (None, None)},
 '214143': {'265': (0, None),
  '나이키': (4, None),
  '새상품': (21, None),
  '에어맥스': (0, None),
  '운동화': (0, None)},
 '214790': {'김종국양말': (None, None),
  '나이키': (11, 12),
  '나이키 조던': (3, 2),
  '나이키양말': (10, 7),
  '스포츠양말': (7, 8),
  '아디다스': (6, 4),
  '아디디스양말': (None, None),
  '언더아머': (6, 7),
  '장목양말': (None, None),
  '조던': (4, 4),
  '크루양말': (None, None)},
 '214791': {'꼼데가르송': (1, 2),
  '나이키': (13, 17),
  '나이키양말': (12, 12),
  '라코스테': (2, 1),
  '라코스테양말': (None, None),
  '아디다스': (8, 7)},
 '214994': {'애플워치': (None, None),
  '애플워치 미개봉': (None, None),
  '애플워치3': (None, None),
  '애플워치4': (None, None),
  '애플워치5': (None, None),
  '애플워치6': (None, None),
  '애플워치6 44': (None, None),
  '애플워치6 나이키': (None, None),
  '애플워치6 미개봉': (None, None),
  '애플워치se': (None, None),
  '애플워치se 40': (None, None),
  '애플워치se 44': (None, None),
  '애플워치se 나이키': (None, None),
  '애플워치se 미개봉': (None, None),
  '애

In [143]:
get_rank('아디다스')

{'213604': {'운동화': (1, None)},
 '214184': {'오즈위고': (None, None)},
 '214773': {'FV3330': (None, None),
  '슈퍼스타': (0, None),
  '아디다스': (1, None),
  '아디다스슈퍼스타': (None, None),
  '아디다스운동화': (None, None)},
 '214790': {'김종국양말': (None, None),
  '나이키': (11, 12),
  '나이키 조던': (3, 2),
  '나이키양말': (10, 7),
  '스포츠양말': (7, 8),
  '아디다스': (6, 4),
  '아디디스양말': (None, None),
  '언더아머': (6, 7),
  '장목양말': (None, None),
  '조던': (4, 4),
  '크루양말': (None, None)},
 '214791': {'꼼데가르송': (1, 2),
  '나이키': (13, 17),
  '나이키양말': (12, 12),
  '라코스테': (2, 1),
  '라코스테양말': (None, None),
  '아디다스': (8, 7)},
 '214792': {'나이키스우시': (None, None),
  '나이키스우시양말': (None, None),
  '나이키양말': (3, 1),
  '나이키컬러양말': (None, None),
  '레깅스양말': (None, None),
  '스포츠양말': (1, 1),
  '아디다스': (7, 5),
  '아디다스양말': (None, None),
  '언더아머': (5, 5),
  '운동양말': (None, None),
  '조던': (5, 6),
  '챔피온': (1, 3),
  '컬러양말': (None, None),
  '크루양말': (None, None)}}

In [144]:
get_rank('샤넬')

{'214477': {'명품가방': (0, 0),
  '명품매입': (4, 3),
  '몽클레어': (1, 1),
  '몽클레어 패딩': (None, None),
  '발렌시아가': (2, 1),
  '샤넬': (5, 4),
  '스톤아일랜드': (0, 1),
  '스톤아일랜드 패딩': (None, None),
  '없음': (None, None),
  '패딩': (1, 1)},
 '214816': {'미니크로스백': (None, None),
  '샤넬': (3, None),
  '샤넬미니백': (None, None),
  '샤넬크로스백': (None, None),
  '샤넬토트백': (None, None),
  '토트백': (0, None)},
 '214842': {'구찌': (2, 1), '빈스': (None, None)}}

In [165]:
get_rank('구찌')

{'214038': {'구찌': (1, None), '버버리': (0, None), '프라다': (0, None)},
 '214111': {'14k': (7, 6)},
 '214608': {'간절기스카프': (None, None),
  '구찌스카프': (1, None),
  '버버리스카프': (None, None),
  '봄스카프': (None, None),
  '샤넬스카프': (None, None),
  '스카프': (4, None),
  '실크스카프': (2, None),
  '에르메스스카프': (1, None),
  '포인트스카프': (None, None)},
 '214817': {'구찌': (6, None),
  '구찌파우치': (None, None),
  '파우치': (2, None),
  '화장품파우치': (None, None),
  '후레임파우치': (None, None)},
 '214842': {'구찌': (2, 1), '빈스': (None, None)},
 '214969': {'까르띠에': (None, None), '오메가': (None, None)}}

In [166]:
get_rank('맨투맨')

{'214361': {'dime': (None, None),
  'dime맨투맨': (None, None),
  'm사이즈': (2, 1),
  '다임': (None, None),
  '다임맨투맨': (None, None),
  '맨투맨': (1, 2)},
 '214537': {'나스카레이싱': (None, None),
  '레이싱맨투맨': (None, None),
  '맨투맨': (2, None),
  '맨투맨티셔츠': (None, None),
  '빈티지맨투맨': (None, None)}}

In [145]:
get_rank('가방')

{'213437': {'가내스라': (0, 0),
  '닥스': (0, 0),
  '닥스가방': (None, None),
  '미뉴에트': (0, 0),
  '미닛뮤트': (None, None),
  '숄더백': (1, 0),
  '시엔느': (0, 0),
  '토리버치': (2, 0)},
 '214209': {'발리': (None, None),
  '발리백': (None, None),
  '발리숄더': (None, None),
  '발리여자가방': (None, None),
  '발리정품': (None, None),
  '숄더백': (0, None)},
 '214309': {'가죽가방': (None, None),
  '도트백': (None, None),
  '메트로시티': (0, 0),
  '숄더백체인백': (None, None),
  '직장인가방': (None, None)},
 '214477': {'명품가방': (0, 0),
  '명품매입': (4, 3),
  '몽클레어': (1, 1),
  '몽클레어 패딩': (None, None),
  '발렌시아가': (2, 1),
  '샤넬': (5, 4),
  '스톤아일랜드': (0, 1),
  '스톤아일랜드 패딩': (None, None),
  '없음': (None, None),
  '패딩': (1, 1)},
 '214627': {'페라가모': (None, None),
  '페라가모가방': (None, None),
  '페라가모숄더백': (None, None),
  '페라가모정품': (None, None),
  '페라가모토트백': (None, None)},
 '214631': {'루이비통': (0, None)},
 '214781': {'가방': (0, None),
  '백팩': (0, None),
  '이스트백': (None, None),
  '책가방': (None, None),
  '프로바이더': (None, None)},
 '214812': {'입생로랑': (3, None),
  '입생로랑가방': (None, N

In [147]:
get_rank('컴퓨터')

{'206440': {'3070': (1, None)},
 '213905': {'pc': (22, 22),
  '데스크탑': (46, 38),
  '롤': (34, 28),
  '메이플스토리': (21, 20),
  '문서작업': (None, None),
  '본체': (36, 29),
  '사무실': (None, None),
  '사무실본체': (None, None),
  '사무용': (None, None),
  '사무용pc': (None, None),
  '사무용본체': (1, 1),
  '사무용컴퓨터': (2, 1),
  '사무컴퓨터': (None, None),
  '서든': (25, 24),
  '웹서핑': (None, None),
  '조립식컴퓨터': (10, 10),
  '컴퓨터': (54, 37),
  '컴퓨터본체': (30, 24),
  '컴퓨터추천': (9, 8),
  '피파': (38, 27)},
 '213906': {'i7': (15, 15),
  'i71700라이젠': (0, 0),
  'pc': (15, 15),
  '게이밍PC': (6, 7),
  '게임용컴퓨터': (11, 13),
  '녹스': (1, 1),
  '데스크탑본체컴퓨터': (1, 3),
  '라이젠': (5, 6),
  '롤': (27, 21),
  '리니지방송용GTX': (4, 4),
  '매크로': (None, None),
  '메이플스토리': (16, 16),
  '모모플레이어': (1, 1),
  '방송용컴퓨터': (8, 10),
  '배그': (35, 28),
  '배틀그라운드': (22, 21),
  '배틀그라운드배그': (4, 5),
  '본체': (20, 14),
  '사무용컴퓨터': (7, 9),
  '사무용컴퓨터조립식컴퓨터': (None, None),
  '오버워치피파롤서든': (5, 5),
  '작업장컴퓨터': (None, None),
  '조립PC': (9, 10),
  '컴퓨터': (38, 33),
  '컴퓨터본체': (25, 20),
  '피파'

In [149]:
get_rank('새상품')

{'213415': {'돌체앤가바나': (0, None),
  '명품': (4, None),
  '새상품': (13, None),
  '선글라스': (None, None),
  '정품': (4, None)},
 '213546': {'고기불판': (None, None),
  '그릴': (2, 1),
  '멀티그릴팬': (None, None),
  '새상품': (24, 18),
  '집들이선물': (None, None),
  '후라이팬': (2, 2)},
 '213961': {'새상품': (43, 32),
  '얼티메이트슬리커': (None, None),
  '플러쉬퍼피': (2, 2),
  '플러쉬퍼피브러쉬': (None, None),
  '플러쉬퍼피슬리커': (None, None)},
 '213964': {'강아지': (3, 4),
  '강아지빗': (None, None),
  '새상품': (42, 31),
  '슬리커브러쉬': (None, None),
  '플러쉬퍼피': (1, 1),
  '플러쉬퍼피브러쉬': (None, None),
  '플러쉬퍼피슬리커': (None, None)},
 '214107': {'게이밍 컴퓨터': (15, 14),
  '데스크탑': (14, 8),
  '본체': (13, 7),
  '컴퓨터': (19, 13),
  '컴퓨터 세트': (None, None),
  '컴퓨터 케이스': (None, None)},
 '214143': {'265': (0, None),
  '나이키': (4, None),
  '새상품': (21, None),
  '에어맥스': (0, None),
  '운동화': (0, None)},
 '214185': {'블레이저': (0, None),
  '블레이져': (0, None),
  '새상품': (1, None),
  '자켓': (0, None)},
 '214186': {'새상품': (20, 12), '심슨': (None, None), '와인오프너': (None, None)},
 '214271': {'가죽바지': 

In [150]:
get_rank('갤럭시')

{'213402': {'S21': (None, None),
  'S21미개봉': (None, None),
  'S21풀박스': (None, None),
  '갤럭시S21풀박스': (None, None),
  '갤럭시s21': (8, 9)},
 '213403': {'S21플러스': (None, None),
  '갤럭시S21플러스': (None, None),
  '갤럭시s21': (9, 8)},
 '213404': {'S20U': (None, None),
  'S20U그레이': (None, None),
  'S20ultra': (None, None),
  's20울트라': (1, 1),
  '갤럭시S20U': (None, None),
  '갤럭시s20울트라': (5, 3)},
 '213409': {'S105G': (4, 3),
  'S105G크라운': (None, None),
  's10': (1, 3),
  's10 5g': (4, 3),
  's105g': (4, 3),
  's105g실버': (None, None),
  '갤럭시 s10': (9, 8),
  '갤럭시S105G': (4, 5)},
 '213410': {'S105G': (3, 7),
  'S105G블랙': (None, None),
  's10': (0, 4),
  's10 5g': (3, 7),
  's105g': (3, 7),
  '갤럭시 s10': (8, 9),
  '갤럭시S105G': (1, 2),
  '갤럭시s105g': (1, 2)},
 '213411': {'S105G': (9, 10),
  'S105G실버': (None, None),
  's10': (2, 5),
  's10 5g': (9, 10),
  's105g': (9, 10),
  '갤럭시 s10': (10, 10),
  '갤럭시S105G': (2, 3),
  '갤럭시s105g': (2, 3)},
 '213443': {'i5': (0, 0),
  '갤럭시북': (None, None),
  '갤럭시북 플렉스': (None, Non

In [155]:
get_rank('에어팟')

{'214339': {'다꾸재료': (None, None),
  '다이어리': (0, None),
  '다이어리키링': (None, None),
  '스트랩': (None, None),
  '에어팟': (2, None),
  '에어팟키링': (None, None),
  '자동차': (None, None),
  '자동차키링': (None, None),
  '키링': (3, None)},
 '214340': {'다꾸': (3, None),
  '다이어리': (1, None),
  '버즈': (None, None),
  '씰스': (None, None),
  '에어팟': (1, None),
  '키링': (2, None)}}

In [167]:
get_rank('배그')

{'206440': {'3070': (1, None)},
 '213905': {'pc': (22, 22),
  '데스크탑': (46, 38),
  '롤': (34, 28),
  '메이플스토리': (21, 20),
  '문서작업': (None, None),
  '본체': (36, 29),
  '사무실': (None, None),
  '사무실본체': (None, None),
  '사무용': (None, None),
  '사무용pc': (None, None),
  '사무용본체': (1, 1),
  '사무용컴퓨터': (2, 1),
  '사무컴퓨터': (None, None),
  '서든': (25, 24),
  '웹서핑': (None, None),
  '조립식컴퓨터': (10, 10),
  '컴퓨터': (54, 37),
  '컴퓨터본체': (30, 24),
  '컴퓨터추천': (9, 8),
  '피파': (38, 27)},
 '213906': {'i7': (15, 15),
  'i71700라이젠': (0, 0),
  'pc': (15, 15),
  '게이밍PC': (6, 7),
  '게임용컴퓨터': (11, 13),
  '녹스': (1, 1),
  '데스크탑본체컴퓨터': (1, 3),
  '라이젠': (5, 6),
  '롤': (27, 21),
  '리니지방송용GTX': (4, 4),
  '매크로': (None, None),
  '메이플스토리': (16, 16),
  '모모플레이어': (1, 1),
  '방송용컴퓨터': (8, 10),
  '배그': (35, 28),
  '배틀그라운드': (22, 21),
  '배틀그라운드배그': (4, 5),
  '본체': (20, 14),
  '사무용컴퓨터': (7, 9),
  '사무용컴퓨터조립식컴퓨터': (None, None),
  '오버워치피파롤서든': (5, 5),
  '작업장컴퓨터': (None, None),
  '조립PC': (9, 10),
  '컴퓨터': (38, 33),
  '컴퓨터본체': (25, 20),
  '피파'

In [164]:
get_rank('애슐리상점')

{}

In [163]:
keywords[keywords['keyword'] == '.']

,keyword,bid_price,ad_set_product_id,ad_set_id,uid
1658,.,50,214831,77922,6584274
1663,.,50,214889,77922,6584274
1668,.,50,214907,77922,6584274
1694,.,50,214826,77922,6584274
1695,.,50,214835,77922,6584274
1701,.,50,214895,77922,6584274
1707,.,50,214902,77922,6584274
1721,.,50,214828,77922,6584274
1731,.,50,214899,77922,6584274
1745,.,50,214891,77922,6584274


In [190]:
# 0325
ad_set_product_ids = keywords2.ad_set_product_id.unique().tolist()
total_num = len(ad_set_product_ids)
print('total length: {}'.format(total_num))
df_pre = {}
for num, i in enumerate(ad_set_product_ids):
    df_pre[i] = {}
    if num % 500 == 0:
        print(num)
    bid_keywords = keywords[keywords['ad_set_product_id'] == str(i)]['keyword'].unique()
    for keyword in bid_keywords:
        results = get_data(keyword)
        for rank, ad in enumerate(results):
            if str(ad['adSetProductId']) == str(i):
                df_pre[i][keyword]=rank
                break

total length: 4066
0
500
1000
1500
2000
2500
3000
3500
4000


In [195]:
import json
with open('pre_data.json', 'w', encoding='UTF-8-sig') as file:
     file.write(json.dumps(df_pre, ensure_ascii=False))

In [198]:
df_pre2 = {}
for product_id in df_pre:
    if len(df_pre[product_id]) > 0:
        df_pre2[product_id] = df_pre[product_id]

In [200]:
len(df_pre2)

487

In [202]:
# 0325
ad_set_product_ids = keywords3.ad_set_product_id.unique().tolist()
total_num = len(ad_set_product_ids)
print('total length: {}'.format(total_num))
df_post = {}
for num, i in enumerate(ad_set_product_ids):
    df_post[i] = {}
    if num % 500 == 0:
        print(num)
    bid_keywords = keywords[keywords['ad_set_product_id'] == str(i)]['keyword'].unique()
    for keyword in bid_keywords:
        results = get_data(keyword)
        for rank, ad in enumerate(results):
            if str(ad['adSetProductId']) == str(i):
                df_post[i][keyword]=rank
                break

total length: 4045
0
500
1000
1500
2000
2500
3000
3500
4000


In [203]:
with open('post_data.json', 'w', encoding='UTF-8-sig') as file:
     file.write(json.dumps(df_post, ensure_ascii=False))
        
df_post2 = {}
for product_id in df_post:
    if len(df_post[product_id]) > 0:
        df_post2[product_id] = df_post[product_id]

In [204]:
len(df_post2)

465

In [206]:
both = []
for i in df_pre2:
    for j in df_post2:
        if i == j:
            both.append(i)
both = list(set(both))

In [207]:
len(both)

383

In [208]:
compare = {}
for product_id in both:
    compare[product_id] = {}
    for keyword in df_pre2[product_id]:
        pre_rank = df_pre2[product_id][keyword]
        if keyword in df_post2[product_id]:
            post_rank = df_post2[product_id][keyword]
        else:
            post_rank = None
        compare[product_id][keyword] = [pre_rank, post_rank, pre_rank-post_rank]

In [210]:
with open('compare_1hr.json', 'w', encoding='UTF-8-sig') as file:
     file.write(json.dumps(compare, ensure_ascii=False))


In [211]:
compare

{194447: {'도손': [0, 0, 0],
  '라튤립': [0, 0, 0],
  '몽파리': [0, 0, 0],
  '블랑쉬': [0, 0, 0],
  '블랙베리앤베이': [0, 0, 0],
  '블루밍부케': [0, 0, 0],
  '오드우드': [1, 1, 0],
  '우드세이지': [0, 0, 0],
  '탐다오': [0, 0, 0],
  '테싯': [0, 0, 0],
  '화이트스웨이드': [1, 1, 0]},
 211791: {'레이스스커트': [0, 0, 0], '빈티지': [0, 0, 0], '쉬폰스커트': [0, 0, 0]},
 213392: {'강력드라이기': [0, 0, 0],
  '강력드라이어': [0, 0, 0],
  '드라이기': [1, 1, 0],
  '드라이기미개봉': [0, 0, 0],
  '드라이기새상품': [0, 0, 0],
  '드라이기추천': [0, 0, 0],
  '드라이어': [0, 0, 0],
  '드라이어새상품': [0, 0, 0],
  '헤어드라이기': [1, 1, 0],
  '헤어드라이어': [0, 0, 0]},
 213393: {'과일주스': [0, 0, 0],
  '녹즙기': [0, 0, 0],
  '야채주스': [0, 0, 0],
  '원액기': [0, 0, 0],
  '주서기': [0, 0, 0],
  '쥬서기': [0, 0, 0],
  '착즙기': [0, 0, 0]},
 213394: {'MVP-6000': [0, 0, 0],
  '공기압마사지': [0, 0, 0],
  '눈마사지': [0, 0, 0],
  '눈마사지기': [0, 0, 0],
  '눈안마': [0, 0, 0],
  '눈안마기': [0, 0, 0],
  '메디니스 눈마사지기': [0, 0, 0],
  '메디니스 눈안마기': [0, 0, 0]},
 213396: {'다지기': [0, 0, 0],
  '대용량믹서기': [0, 0, 0],
  '믹서기': [2, 2, 0],
  '분쇄기': [0, 0, 0],
  '블랜더': [1, 0, 

### 3월 25일 10시 50분 경
#### pre[bid_keyword] = {rank: [ad set id, ad set product id, bid price]}

In [130]:
pre = dict()
for key in top_keywords:
    data = get_data(key)
    tmp = {}
    for idx, ad in enumerate(data):
        tmp[idx] = [ad['adSetId'], ad['adSetProductId'], ad['bidPrice']]
    pre[key] = tmp

In [132]:
pre['아이폰12']

{0: [75423, 210370, 300],
 1: [77869, 214734, 300],
 2: [76148, 211954, 300],
 3: [74498, 208013, 250],
 4: [77390, 213955, 200],
 5: [77464, 214099, 95],
 6: [75097, 209545, 85],
 7: [75092, 209522, 85],
 8: [75097, 209551, 85],
 9: [75097, 209556, 85],
 10: [75097, 209544, 85],
 11: [75423, 210373, 60],
 12: [75423, 210371, 60],
 13: [75768, 211162, 100],
 14: [78053, 215094, 50],
 15: [78053, 215095, 50],
 16: [75109, 209638, 50],
 17: [75519, 210573, 50],
 18: [75109, 209643, 50],
 19: [75109, 209644, 50],
 20: [75519, 210569, 50],
 21: [75109, 209635, 50],
 22: [75109, 209636, 50],
 23: [77831, 214672, 50],
 24: [75109, 209642, 50],
 25: [75109, 209637, 50],
 26: [75109, 209640, 50],
 27: [75109, 215099, 50],
 28: [75111, 209659, 50],
 29: [75109, 209647, 50],
 30: [75109, 209634, 50],
 31: [75109, 209641, 50],
 32: [75109, 209631, 50],
 33: [75092, 209523, 50],
 34: [78053, 215096, 50],
 35: [75092, 209521, 50],
 36: [75109, 209632, 50],
 37: [75109, 209645, 50],
 38: [75097, 209

### 3월 24일 14시 50분 경

In [139]:
post = dict()
for key in top_keywords:
    data = get_data(key)
    tmp = {}
    for idx, ad in enumerate(data):
        tmp[idx] = [ad['adSetId'], ad['adSetProductId'], ad['bidPrice']]
    post[key] = tmp

In [29]:
post['아이폰12']

{0: [75423, 210370, 305],
 1: [77390, 213955, 250],
 2: [76205, 212135, 300],
 3: [77464, 214099, 210],
 4: [76148, 211954, 300],
 5: [74498, 208013, 250],
 6: [75092, 209522, 85],
 7: [75097, 209545, 85],
 8: [75097, 209551, 85],
 9: [75097, 209556, 85],
 10: [75097, 209544, 85],
 11: [75768, 211162, 100],
 12: [75423, 210371, 60],
 13: [75423, 210373, 60],
 14: [75109, 209638, 50],
 15: [75109, 209644, 50],
 16: [75109, 209636, 50],
 17: [75109, 209643, 50],
 18: [75109, 209635, 50],
 19: [75519, 210573, 50],
 20: [75519, 210569, 50],
 21: [75109, 209647, 50],
 22: [75109, 209642, 50],
 23: [75109, 209640, 50],
 24: [75109, 209637, 50],
 25: [75111, 209659, 50],
 26: [75109, 209641, 50],
 27: [75109, 209634, 50],
 28: [75109, 209632, 50],
 29: [75109, 209631, 50],
 30: [75092, 209521, 50],
 31: [75109, 209645, 50],
 32: [75092, 209523, 50],
 33: [75109, 209633, 50],
 34: [75097, 209565, 50],
 35: [75097, 209560, 50],
 36: [75097, 209573, 50],
 37: [75097, 209562, 50],
 38: [75097, 20

In [32]:
def compare(keyword):
    pre_dict = {}
    post_dict = {}
    for key, value in pre[keyword].items():
        pre_dict[value[1]] = key
    for key, value in post[keyword].items():
        post_dict[value[1]] = key
        
    compare_dict = {}
    for p in pre_dict:
        if p in post_dict:
            compare_dict[p] = int(pre_dict[p]) - int(post_dict[p])
    return compare_dict

In [33]:
iphone12 = compare('아이폰12')

In [35]:
iphone12

{208013: -2,
 209518: 0,
 209519: 0,
 209520: 0,
 209521: 0,
 209522: -1,
 209523: 0,
 209544: -2,
 209545: -1,
 209546: -2,
 209547: 0,
 209548: 0,
 209549: 1,
 209550: 0,
 209551: -1,
 209552: 0,
 209553: 0,
 209554: -2,
 209555: 61,
 209556: 0,
 209557: 1,
 209558: 1,
 209559: 0,
 209560: 1,
 209561: 0,
 209562: -57,
 209564: 1,
 209565: 0,
 209566: 0,
 209572: 1,
 209573: -1,
 209574: 1,
 209631: 0,
 209632: 0,
 209633: 0,
 209634: -1,
 209635: -2,
 209636: -1,
 209637: 0,
 209638: -1,
 209640: -1,
 209641: 1,
 209642: -1,
 209643: 0,
 209644: -1,
 209645: 0,
 209647: 2,
 209659: 0,
 210370: 0,
 210371: -1,
 210373: -1,
 210568: -4,
 210569: -1,
 210572: 0,
 210573: -1,
 210574: -1,
 211162: -1,
 211954: -2,
 212135: -1,
 213586: -1,
 213955: 3}